# Welcome to the DYAD component of the Flux tutorial


> What is DYAD? 🤔️

DYAD is a transparent, locality-aware, write-once, read-many file cache that runs on top of local NVMe and other burst buffer-style technologies (e.g., El Capitan Rabbit nodes). It is designed to accelerate large, distributed workloads, such as distributed Deep Learning (DL) training and scientific computing workflows, on HPC systems. It is also designed be transparent, which allows users to leverage DYAD with little to no code refactoring. Unlike similar tools (e.g., DataSpaces and UnifyFS), which tend to optimize for write performance, DYAD aims to provide good write **and read** performance. To optimize read performance, DYAD uses a locality-aware "Hierarchical Data Locator," which prioritizes node-local metadata and data retrieval to minimize the amount of network communications. When moving data from another node, DYAD also uses a streaming RPC over RDMA protocol, which uses preallocated buffers and connection caching to maximize network bandwidth. This process is shown in the figure below:

![DYAD Reading Process](img/dyad_design.png)

DYAD uses several services provided by Flux (key-value store, remote proceedure call, broker modules) to orchestrate data movement between nodes. It also uses UCX to move data.

> I'm ready! How do I do this tutorial? 😁️

The process for running this tutorial is the same as `flux.ipynb`. To step through examples in this notebook 
you need to execute cells. To run a cell, press Shift+Enter on your keyboard. If you prefer, you can also paste 
the shell commands in the JupyterLab terminal and execute them there.

# Accelerating Distributed Deep Learning (DL) Training with DYAD

Description of distributed DL (from DLIO paper)

## Leveraging DYAD in PyTorch

When using custom datasets or custom techniques/tools to read a dataset from storage, PyTorch requires the creation of `Dataset` and `DataLoader` classes. To use DYAD in PyTorch-based distributed DL training, we have implemented several of these classes. They can all be found in the [dlio_extensions](../dlio_extensions) directory.

The specific classes used in this tutorial are `DYADTorchDataset` and `DyadTorchDataLoader`, which can both be found [here](../dlio_extensions/dyad_torch_data_loader.py). The `DYADTorchDataset` class contains all the DYAD-specific code. The `DyadTorchDataLoader` class is a basic `DataLoader` designed to read individual samples from the `DYADTorchDataset` class.

In the following code cells, we show the DYAD-specific code in `DYADTorchDataset`. As you will see, this code is very similar to standard Python file I/O. As a result, this code serves as an example of DYAD's transparency.

<div class="alert alert-block alert-info">
<b>Note:</b> due to several aspects of PyTorch's design (described below), DYAD cannot be used as transparently as normal. Normally, in Python, users would just have to replace the built-in `open` function for DYAD's `dyad_open`. As a result, this use case should be considered the *worst case* for DYAD's transparency.
</div>

In [ ]:
import os
import sys
import inspect
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML

sys.path.insert(0, os.path.abspath("../dlio_extensions/"))

from dyad_torch_data_loader import DYADTorchDataset

This first block of code shows the `DYADTorchDataset.worker_init` function. This function is called to initialize the I/O processes that are spawned by the `DyadTorchDataLoader` class. As a result, this function contains two parts: (1) the initialization of PyTorch internals and utilities (e.g., a logger) and (2) the initialization of DYAD.

Normally, DYAD is configured using environment variables, and, as a result, DYAD's initialization can be hidden from users. However, due to PyTorch's complexity and challenges in correctly propagating environment variables through PyTorch's dynamic process spawning, DYAD's transparent, environment variable-based initialization cannot be used in `DYADTorchDataset`. Instead, we manually initialize and configure DYAD using `Dyad.init()`.

In [ ]:
display(HTML(highlight(inspect.getsource(DYADTorchDataset.worker_init), PythonLexer(), HtmlFormatter(full=True))))

This second block of code shows the `DYADTorchDataset.__getitem__` function. This function is called by `DyadTorchDataLoader` to read individual samples for a batch from disk. With other `Dataset` classes, this function would simply identify the file containing the requested sample and read that sample from remote storage (e.g., Lustre) using Python's built-in `open` function. On the other hand, `DYADTorchDataset` does four things. First, it identifies the file containing the requested sample. Second, it uses DYAD's `get_metadata` function to check if that file has already been cached into DYAD. Third, if the file has already been cached, it will retrieve the sample using DYAD's `dyad_open` function. This function retrieves the sample from a different node, if needed, and then makes that sample available through an interface equivalent to Python's built-in `open` function. Finally, if the file has **not** been cached, it will read the sample from remote storage (e.g., Lustre) and cache the sample into DYAD for more efficient future reading.

In [ ]:
display(HTML(highlight(inspect.getsource(DYADTorchDataset.__getitem__), PythonLexer(), HtmlFormatter(full=True))))

## Configure DLIO and DYAD

Now that we've seen how DYAD is integrated into PyTorch, we will start configuring DYAD and DLIO.

First, we will configure DYAD. DYAD requires three settings for configuration:
1. A namespace in the Flux key-value store, which DYAD will use for metadata management
2. A "managed directory," which DYAD will use to determine the files that should be tracked
3. A data transport layer (DTL) mode, which DYAD will use to select the underlying networking library for data transfer 

In [ ]:
kvs_namespace = "dyad"
managed_directory = "./dyad_data"
dtl_mode = "UCX" # We currently only support UCX, so do not change this

Next, we will configure DLIO. DLIO requires several configuration settings. However, for this tutorial, the only one that should be set is the initial data directory, or the directory where the dataset initially resides at the start of training. When running DLIO, the `DYADTorchDataset` class will dynamically copy files from this directory into DYAD's managed directory.

In [ ]:
initial_data_directory = "./dlio_data"

Finally, we will set the remaining configurations for DLIO. These should not be edited because they depend on the directory structure and configuration of this tutorial.

In [ ]:
workers_per_node = 1
dyad_install_prefix = "/usr/local"
num_nodes = 2
dlio_extensions_dir = "/home/jovyan/flux-tutorial-2024/dlio_extensions"
workload = "dyad_unet3d_demo"

To properly set the environment variables needed for running DLIO with DYAD, we will create an environment file that is compatible with the `--env-file` flag of `flux submit`.

In [ ]:
env_file = f"""
DYAD_KVS_NAMESPACE={kvs_namespace}
DYAD_DTL_MODE={dtl_mode}
DYAD_PATH={managed_directory}
PYTHONPATH={dlio_extensions_dir}:$PYTHONPATH
DLIO_PROFILER_ENABLE=0
"""

with open("dlio_env.txt", "w") as f:
    f.write(env_file)

## Create Flux KVS Namespace and start DYAD service

Next, we will start the DYAD service. This involves two steps. First, we need to create a namespace withing the Flux key-value store. This namespace will be used by DYAD to store metadata about cached files. This metadata is then used by DYAD's Hierarchical Data Locator to locate files.

In [ ]:
!flux kvs namespace create {kvs_namespace}

After creating the key-value store namespace, we will start the DYAD service. The DYAD service is implemented as a Flux broker module. This allows us to use Flux for service deployment and application-to-service communication. To start the DYAD service, we use the `flux module load` command. We run that command through `flux exec -r all` to deploy the service across all Flux brokers.

In [ ]:
!flux exec -r all flux module load {dyad_install_prefix}/lib/dyad.so --mode={dtl_mode} {managed_directory}

Finally, we can check that the service and key-value store namespace were successfully created with the cells below.

In [ ]:
!flux module list

In [ ]:
!flux kvs namespace list

## Generate Data for Unet3D

Before running DLIO, we need to obtain data for the unet3d use case. Instead of downloading the full dataset, we will use DLIO to generate a smaller, synthetic version of the dataset for this tutorial.

In [ ]:
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {managed_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {managed_directory}/* 
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {initial_data_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {initial_data_directory}/* 

In [ ]:
!flux run -N {num_nodes} -o cpu-affinity=off --tasks-per-node={workers_per_node} --env-file=dlio_env.txt \
    dlio_benchmark --config-dir={dlio_extensions_dir}/configs workload={workload} \
        ++workload.dataset.data_folder={initial_data_directory} ++workload.workflow.generate_data=True \
        ++workload.workflow.train=False

## Run "training" through DLIO

Now, we will run DLIO using the command below. As DLIO runs, it will print out logging statements showing how long sample reading is taking. At the end, DLIO will print out a performance summary of the run.

In [ ]:
!flux run -N {num_nodes} -o cpu-affinity=on --tasks-per-node={workers_per_node} --env-file=dlio_env.txt \
    dlio_benchmark --config-dir={dlio_extensions_dir}/configs workload={workload} \
        ++workload.dataset.data_folder={initial_data_directory} ++workload.workflow.generate_data=False \
        ++workload.workflow.train=True

## Shutdown the DYAD service and cleanup

Now that we are done running DLIO, we need to shutdown the DYAD service and remove the key-value store namespace used by DYAD. This can be done with the two Flux commands below.

In [ ]:
!flux kvs namespace remove {kvs_namespace}
!flux exec -r all flux module remove dyad

The following cells show that the DYAD service has been removed and that the namespace has been removed from the key-value store.

In [ ]:
!flux module list

In [ ]:
!flux kvs namespace list

Finally, we need to remove all the files we generated while running DLIO. We use `flux run` to ensure that any node-local files are deleted.

In [ ]:
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {managed_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {managed_directory}/* 
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {initial_data_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {initial_data_directory}/* 

# Full Scale Results

# This concludes the notebook tutorial for DYAD.

If you are interested in learning more about DYAD, check out our [ReadTheDocs page](https://dyad.readthedocs.io/en/latest/), our [GitHub repository](https://github.com/flux-framework/dyad), and our published/presented works:
* [eScience 2022 Short Paper](https://dyad.readthedocs.io/en/latest/_downloads/27090817b034a89b76e5538e148fea9e/ShortPaper_2022_eScience_LLNL.pdf)
* [SC 2023 ACM Student Research Competition Extended Abstract](https://github.com/flux-framework/dyad/blob/main/docs/_static/ExtendedAbstract_2023_SC_ACM_SRC_DYAD.pdf)
* [IPDPS 2024 HiCOMB Workshop Paper](https://github.com/flux-framework/dyad/blob/main/docs/_static/Paper_2024_IPDPS_HiCOMB_DYAD.pdf)

If you are interested in working with us, please reach out to Jae-Seung Yeom (yeom2@llnl.gov), Hariharan Devarajan (hariharandev1@llnl.gov), or Ian Lumsden (ilumsden@vols.utk.edu).